In [ ]:
import pandas as pd
from global_modules import Sqlite
import os
import sys, getopt
import time

def main(argv):
    opts, args = getopt.getopt(argv, "hl:")
    for opt, arg in opts:
        if opt == '-h':
            print('-l <label>')
            sys.exit()
        elif opt in ('-l'):
            return arg
        else:
            print('Please specify label parameter first')
            sys.exit()
            
if __name__ == '__main__':
    
    label = main(sys.argv[1:])
    # print(sys.argv[1:])
    # print(label)

    root_path = os.getcwd()
    db_path = os.path.join(root_path, 'db')
    movement_signal_db_path = os.path.join(db_path, 'movement_signal.db')
    cache_path = os.path.join(root_path, 'cache')
    movements_path = os.path.join(cache_path, 'movements')
    thresholds_path = os.path.join(movements_path, 'thresholds')
    thresholds_label_path = os.path.join(thresholds_path, label)
    os.makedirs(thresholds_label_path, exist_ok=True)
    
    # Constants
    capture_rate = 1.65 #seconds
    data_limit = int(24 * 60 * 60 / capture_rate) # 24hrs * 60mins * 60secs, a day in seconds
    
    # Database initialization
    sqlite = Sqlite()
    sqlite.set_table(label)
    sqlite.create_connection(movement_signal_db_path)
    
    # Processing threshold
    try:
        while True:
            df = pd.read_sql(f'select * from {label} order by ROWID desc limit {data_limit}', sqlite.conn)
            df['value'] = df['value'].astype('int')
            filtered_1std = df.loc[df['value'] > (df['value'].mean() + df['value'].std())]
            threshold = filtered_1std['value'].mean() + filtered_1std['value'].std()
            timestamp = df.iloc[0]['timestamp']

            # Store threshold
            threshold_path = os.path.join(thresholds_label_path, f'{timestamp}.cache')
            with open(threshold_path, 'w') as f:
                f.write(str(threshold))

            # Keep last 10 threshold
            threshold_value_list = os.listdir(thresholds_label_path)
            if len(threshold_value_list) > 10:

                tvl_float = [float(i.replace('.cache','')) for i in threshold_value_list]
                tvl_float.sort(reverse=False) # ascending, oldest value first

                for fn in tvl_float[:len(tvl_float) - 10]:
                    threshold_value_todelete = os.path.join(thresholds_label_path, f'{fn}.cache')
                    if os.path.isfile(threshold_value_todelete):
                        os.remove(threshold_value_todelete)

            print('Waiting for new data...')
            time.sleep(60)
    
    except KeyboardInterrupt:
        print(f'Stopping movement signal threshold server at {label}')
        pass
    
    


In [ ]:

root_path = os.getcwd()
db_path = os.path.join(root_path, 'db')
movement_signal_db_path = os.path.join(db_path, 'movement_signal.db')

# Constants
capture_rate = 1.65 #seconds
data_limit = int(24 * 60 * 60 / capture_rate) # 24hrs * 60mins * 60secs, a day in seconds

# Database initialization
sqlite = Sqlite()
sqlite.set_table(label)
sqlite.create_connection(movement_signal_db_path)

df = pd.read_sql(f'select * from {label} order by ROWID desc limit {data_limit}', sqlite.conn)
df['value'] = df['value'].astype('int')
filtered_1std = df.loc[df['value'] > (df['value'].mean() + df['value'].std())]
threshold = filtered_1std['value'].mean() + filtered_1std['value'].std()
timestamp = df.iloc[0]['timestamp']